## importing and cleaning

In [31]:
import pandas as pd
data=pd.read_csv('spam.csv',names=['soh','text','1','2','3'],engine='python',error_bad_lines=False,encoding = "ISO-8859-1")

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [32]:
data=data.drop(data.iloc[:,[2,3,4]],axis=1)

In [33]:
data.head()

,soh,text
0,v1,v2
1,ham,"Go until jurong point, crazy.. Available only ..."
2,ham,Ok lar... Joking wif u oni...
3,spam,Free entry in 2 a wkly comp to win FA Cup fina...
4,ham,U dun say so early hor... U c already then say...


In [34]:
data.isnull().sum()

soh     0
text    0
dtype: int64

In [35]:
data=data.drop(0)

In [38]:
data.reset_index(inplace=True)

In [41]:
data=data.drop('index',axis=1)

In [42]:
data.head()

,soh,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [49]:
#data cleaning (stopwords and stem or lem)
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('all')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger

True

In [50]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
stemmer=PorterStemmer()
lemmatizer=WordNetLemmatizer()
corpus=[]
for i in range(len(data['text'])):
  review=re.sub('[^a-zA-Z0-9]',' ',data['text'][i])
  review=review.lower()
  review=review.split()
  review=[lemmatizer.lemmatize(words) for words in review if words not in stopwords.words('english')]
  review=' '.join(review)
  corpus.append(review)

In [52]:
corpus[0]

'go jurong point crazy available bugis n great world la e buffet cine got amore wat'

# embedding representation

In [53]:
#one_hot representation
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot
voc_size=5000
onehot_repr=[one_hot(words,voc_size) for words in corpus]

In [55]:
onehot_repr[0]

[2240,
 1530,
 1737,
 2871,
 2846,
 1312,
 4320,
 4624,
 2494,
 2022,
 2477,
 2708,
 4594,
 1617,
 778,
 3723]

In [69]:
#word embedding representation
from tensorflow.keras.layers import Embedding,LSTM,Bidirectional,Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
import numpy as np


In [58]:
max([len(sent) for sent in onehot_repr])

78

In [61]:
sent_length=8
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

In [62]:
embedded_docs

array([[2494, 2022, 2477, ..., 1617,  778, 3723],
       [   0,    0, 4418, ...,  662,  868, 2841],
       [3480, 4244, 3441, ..., 4803, 4852, 1974],
       ...,
       [   0,    0,    0, ..., 3627, 4875, 4419],
       [2873, 2838,  310, ...,  637,  868, 2858],
       [   0,    0,    0, ...,  301, 3221, 1555]], dtype=int32)

In [70]:
#giving feature to each element
feat_dim=30
model=Sequential()
model.add(Embedding(voc_size,feat_dim,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [71]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 8, 30)             150000    
                                                                 
 lstm (LSTM)                 (None, 100)               52400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 202,501
Trainable params: 202,501
Non-trainable params: 0
_________________________________________________________________


In [73]:
len(embedded_docs)

5572

In [83]:
y=pd.get_dummies(data['soh'])

In [85]:
y=y['spam']

In [86]:
#dependent and independent
x=np.array(embedded_docs)
y=np.array(y)

In [87]:
x.shape,y.shape

((5572, 8), (5572,))

In [89]:
#splitting
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y)

In [90]:
model.fit(xtrain,ytrain,validation_data=(xtest,ytest),epochs=10,batch_size=64)

Epoch 1/10
66/66 [==============================] - 4s 31ms/step - loss: 0.3445 - accuracy: 0.8806 - val_loss: 0.1241 - val_accuracy: 0.9706
Epoch 2/10
66/66 [==============================] - 2s 33ms/step - loss: 0.0704 - accuracy: 0.9840 - val_loss: 0.0801 - val_accuracy: 0.9770
Epoch 3/10
66/66 [==============================] - 1s 21ms/step - loss: 0.0254 - accuracy: 0.9931 - val_loss: 0.0780 - val_accuracy: 0.9777
Epoch 4/10
66/66 [==============================] - 1s 18ms/step - loss: 0.0112 - accuracy: 0.9981 - val_loss: 0.1211 - val_accuracy: 0.9727
Epoch 5/10
66/66 [==============================] - 1s 18ms/step - loss: 0.0073 - accuracy: 0.9983 - val_loss: 0.1002 - val_accuracy: 0.9770
Epoch 6/10
66/66 [==============================] - 1s 19ms/step - loss: 0.0065 - accuracy: 0.9988 - val_loss: 0.1082 - val_accuracy: 0.9785
Epoch 7/10
66/66 [==============================] - 1s 19ms/step - loss: 0.0039 - accuracy: 0.9993 - val_loss: 0.1095 - val_accuracy: 0.9770
Epoch 8/10
66

In [99]:
ypred=model.predict(xtest)

44/44 [==============================] - 0s 4ms/step


In [92]:
ypred

array([[2.6113432e-06],
       [1.3163037e-03],
       [2.4287205e-04],
       ...,
       [9.9998385e-01],
       [1.3500963e-03],
       [9.9966550e-01]], dtype=float32)

In [94]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [113]:
ypred_cond=np.where(ypred>0.5,1,0)

In [114]:
confusion_matrix(ytest,ypred_cond)

array([[1186,   21],
       [  19,  167]])

In [115]:
accuracy_score(ytest,ypred_cond)

0.9712849964106246

In [116]:
own_test="TWILLS Bigget Year End Sale is live now. Rush to shop from our trendy new collection added & get Flat 50% offer. Visit now. T&C. bit.ly/3kkodo0"

In [117]:
own_test=re.sub('[^a-zA-Z0-9]',' ',own_test)
own_test=own_test.lower()
own_test=own_test.split()
own_test=[lemmatizer.lemmatize(words) for words in own_test if words not in stopwords.words('english')]
own_test=' '.join(own_test)

In [118]:
own_test

'twill bigget year end sale live rush shop trendy new collection added get flat 50 offer visit c bit ly 3kkodo0'

In [129]:
test_onehot=[one_hot(own_test,voc_size)]

In [130]:
test_onehot

[[3405,
  1609,
  2271,
  1268,
  59,
  3786,
  2535,
  3648,
  1499,
  3411,
  3535,
  4254,
  1050,
  3295,
  3776,
  2007,
  152,
  4803,
  389,
  2997,
  2347]]

In [131]:
embedded_doc_test=pad_sequences(test_onehot,padding='pre',maxlen=sent_length)

In [132]:
embedded_doc_test

array([[3295, 3776, 2007,  152, 4803,  389, 2997, 2347]], dtype=int32)

In [134]:
own_test_res=model.predict(embedded_doc_test)

1/1 [==============================] - 0s 28ms/step


In [135]:
result=np.where(own_test_res>0.5,1,0)

array([[1]])

In [ ]:
if(int(result)):
  print("spam")
else:
  print("not spam")